In [2]:
import json
import requests
import time
import re
import os
import warnings
import pandas as pd
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from bs4 import BeautifulSoup
import urllib


""" 初始化 """
# 忽略不安全連線警告
warnings.simplefilter('ignore', InsecureRequestWarning)

# 使用header偽裝成edge瀏覽器下瀏覽行為
headers = {'content-type': 'text/html; charset = UTF-8', 
           'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36 Edg/95.0.1020.44'
          }


sleep_time = 5

def timer(func):
    def wrapper(*args, **kwargs):
        tic = time.time()
        value = func(*args, **kwargs)
        print('Time consumed: {} sec.'.format(time.time()-tic))
        return value
    return wrapper



class Merchant:
    def __init__(self, name, keyword, page):
        self.name = name
        self.page = page
        self.keyword = keyword
        self.get_html()
        self.parse_attr()
        
    
    def get_html(self):
        url = 'https://google.com/search?q="{}"+{}&start={}'.format(urllib.parse.quote(self.name), self.keyword, (self.page-1)*10)
        count = 0
        
        while count < 3:
            try:
                result = requests.get(url, verify=False, auth=('user', 'pass'), headers=headers)
                if result.status_code == 200:
                    self.html = result.text
                    break
                elif result.status_code == 429:
                    count = count+1
                    time.sleep(10*60)
            except:
                time.sleep(sleep_time)
                count = count+1
        else:
            raise ConnectionError('Cannot get the html')
    
    def parse_attr(self):
        soup = BeautifulSoup(self.html, 'html.parser')
        self.soup = soup
        
        self.top_address = [element.text for element in soup.findAll('div', {'class':'sXLaOe'})]
        
        self.title_list = [element.find('h3').text for element in soup.findAll('div', {'class':'yuRUbf'})]
        self.summary_list = [element.text for element in soup.findAll('div', {'class':'VwiC3b yXK7lf MUxGbd yDYNvb lyLwlc lEBKkf'})]

    
    def to_dict(self, drop_html=True):
    
        result = {
                    'name' : self.name,
                    'top_address' : self.top_address,
                    'title_list' : self.title_list,
                    'summary_list' : self.summary_list,
                 }
        if not drop_html:
            result['html'] = self.html
        return result


In [3]:
fp_output = r'./content/result_{}_{}.json'
fp_job_status = r'./content/job_status_{}_{}.json'

brands = [['石二鍋',[0,1,0,0,1,0,0]],['春水堂',['飲料店',"餐廳"]]]

pages = 5

# INPUT YOUR KEYWORDS
keywords = ['品牌']
sleep_time = 5 #每一頁要隔五秒

start_time = time.time()
tic = time.time()
toc = time.time()

for i, keyword in enumerate(keywords):
    if i != 0:
        time.sleep(30) #大量的話要久一點
    for page in range(1,pages+1,1):

        result = list()
        job_status = {'fail_list' : [], 'success_list' : [], 'todo': [sublist[0] for sublist in brands], 'done_count': 0}

        for brand in job_status['todo']:
            
            if time.time()-tic < sleep_time:
                time.sleep(sleep_time)
                
            tic = time.time()
            
            try:
                result.append(Merchant(brand, keyword, page).to_dict())
                job_status['success_list'].append(brand)
                
            except:
                job_status['fail_list'].append(brand)
            job_status['done_count'] += 1

            if job_status['done_count'] % 100 == 0: 
                
                print('Jobs done: {}\nTime consumed: {:.1f}\nTotal time consumed: {:.1f}'.format(job_status['done_count'], time.time()-toc, time.time()-start_time))
                print('Success count: {} Fail count: {}'.format(len(job_status['success_list']), len(job_status['fail_list'])))
                print('===========================================================================')
                toc = time.time()

        with open(fp_output.format(keyword, page), 'w') as f:
             f.write(json.dumps(result))
        with open(fp_job_status.format(keyword, page), 'w') as f:
            json.dump(job_status, f)
            
        print('Keyword: {}, page: {} done.'.format(keyword, page))
        print('===========================================================================')

{'fail_list': [], 'success_list': [], 'todo': ['石二鍋', '春水堂'], 'done_count': 0}


In [64]:
brands = [['石二鍋',[0,1,0,0,1,0,0]],['春水堂',['飲料店',"餐廳"]]]

# INPUT YOUR KEYWORDS
keywords = ['品牌']
def create_summary_df(keyword_list, brands, pages):
    for keyword in keyword_list:
        for page in range(pages):
            with open(f'./content/result_{keyword}_{page+1}.json') as f:
                data = json.load(f)
            summary_df = pd.DataFrame()
            for brand_index in range(len(brands)):
                tmp_str = ""
                for i in data[brand_index]["summary_list"]:
                    tmp_str = tmp_str + i
                summary_df = summary_df.append({"name":data[brand_index]["name"],"summary":tmp_str,"category": brands[brand_index][1]},ignore_index = True)
    return summary_df

In [65]:
create_summary_df(keywords, brands, pages)

,category,name,summary
0,"[0, 1, 0, 0, 1, 0, 0]",石二鍋,群品餐飲「石二鍋」、「12MINI」品牌專案委託設計案 ... 本創作連續三年（2017、2...
1,"[飲料店, 餐廳]",春水堂,2017年11月8日 — 超過30年歷史的春水堂，結合創新與台式泡沫紅茶文化，以及發明了世界...
